<a href="https://colab.research.google.com/github/prakshiptha07/abc/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [ ]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

## Lex only

In [ ]:
#@title Writing Lex program
%%writefile program.l

%{
    #include <stdio.h>
    int ctChar=0;
    int ctSpace=0;
    int ctWord=0;
    int ctLine=0;
%}
WORD [^ \t\n,\.:]+
EOL [\n]
BLANK [ ]
%%

{WORD} {ctWord++; ctChar+=yyleng;}
{BLANK} {ctSpace++;}
{EOL} {ctLine++;}
. {ctChar++;}
%%

void main(int argc, char *argv[]){
    if(argc!=2){
        printf("Usage:\n\t./a.out <FILENAME>\n");
        exit(0);
    }

    yyin=fopen(argv[1],"r");
    yylex();

    printf("Word Count: %d\n",ctWord);
    printf("Character Count: %d\n",ctChar);
    printf("Space Count: %d\n",ctSpace);
    printf("Line Count: %d\n",ctLine);
    fclose(yyin);

}

int yywrap(){
    return 1;
}

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

Writing program.txt


In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


## Lex and Yacc combined

In [ ]:
%{
#include <stdio.h>
#include <stdlib.h>
int yylex(void);
void yyerror(const char *s);
int valid = 1;
%}
%token ID
%token PLUS STAR LPAREN RPAREN
%%
input:
     | input line
     ;
line: E '\n' { if(valid) printf("Parsing successful.\n"); valid = 1; }
    | error '\n' { valid = 0; yyerror("Invalid expression"); yyclearin; yyerrok; } ;
E : T E1
  ;
E1 : PLUS T E1
   | /* ε */
   ;
T : F T1
  ;
T1 : STAR F T1
   | /* ε */
   ;
F : LPAREN E RPAREN
  | ID
  ;
%%
void yyerror(const char *s) {
 printf("%s\n", s);
}
int main() {
 printf("Enter expression: ");
 yyparse();
 return 0;
}


Overwriting program.y


In [ ]:
%{
#include "y.tab.h"
%}
%%
[ \t]+ ;
\n { return '\n'; }
"(" { return LPAREN; }
")" { return RPAREN; }
"+" { return PLUS; }
"*" { return STAR; }
"id" { return ID; }
[a-zA-Z][a-zA-Z0-9]* { return -1; }
. { return -1; }
%%
int yywrap(void)
{
 return 1;
}

Overwriting program.l


if you want to use at txt as an input

In [ ]:
%%writefile program.txt

This is a sample file.

In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

yacc -d program.y
lex program.l
cc y.tab.c lex.yy.c -ll
./a.out

y.tab.c: In function ‘yyparse’:
y.tab.c:1121:16: warning: implicit declaration of function ‘yylex’ [-Wimplicit-function-declaration]
       yychar = yylex ();
                ^~~~~
y.tab.c:1256:7: warning: implicit declaration of function ‘yyerror’; did you mean ‘yyerrok’? [-Wimplicit-function-declaration]
       yyerror (YY_("syntax error"));
       ^~~~~~~
       yyerrok
Enter the variable name: variable_name
Valid Identifier
